## Анализ базы данных c помощью SQL

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

К нам обратился клиент, чья компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Цель:** проанализировать базу данных. В ней есть информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

В нашем распоряжении 5 таблиц.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задачи

1. Изучение данных;
2. Исследовательский анализ данных:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, таким образом исключив из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг (только книги с 50 и более оценками);
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
3. Общие выводы.

## Изучение данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db']) 
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# настройки отображения
pd.set_option("max_colwidth", 200)
pd.set_option('display.float_format', '{:.2f}'.format)     

In [3]:
# функция для выполнения запроса
def get(query):
    return pd.io.sql.read_sql(query, con = engine)

In [4]:
# выведем общую информацию таблицы books
books = '''
        
SELECT * FROM books;
        
        '''
get(books).info()

# первые строки таблицы books
get(books).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


В нашей базе 1000 уникальных книг на английском языке

In [5]:
# выведем общую информацию таблицы authors
authors = '''
          
SELECT * FROM authors;
          
          '''
get(authors).info()

# первые строки таблицы authors
get(authors).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


636 авторов

In [6]:
# выведем общую информацию таблицы publishers
publishers = '''
             
SELECT * FROM publishers;
             
             '''
get(publishers).info()

# первые строки таблицы publishers
get(publishers).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


340 издательств

In [7]:
# выведем общую информацию таблицы ratings
ratings = '''
          
SELECT * FROM ratings;
          
          '''
get(ratings).info()

# первые строки таблицы ratings
get(ratings).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


6456 оценок

In [8]:
reviews = '''
        
SELECT * FROM reviews;
          
          '''

# выведем общую информацию таблицы reviews
get(reviews).info()

#выведем первые строки таблицы reviews
get(reviews).head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


2793 обзора

### Выводы

В нашей базе:
- 1000 уникальных книг на английском языке;
- от 636 авторов;
- от 340 издательств;
- 6456 оценок, что довольно неплохо для такого объема книг, в среднем по 6 оценок на книгу;
- 2793 обзора.

## Исследовательский анализ данных

### Посчитать, сколько книг вышло после 1 января 2000 года

In [9]:
books = '''
        
SELECT count(book_id) AS books_count
  FROM books
 WHERE publication_date > '01-01-2000';
        
        '''
get(books)

,books_count
0,819


После 1 января 2000 года вышло 819 книг, это 81.9% от общего количества

### Для каждой книги посчитать количество обзоров и среднюю оценку

In [10]:
query = '''
  
  SELECT books.title AS title,
         avg(ratings.rating) AS avg_rating,
         count(DISTINCT reviews.text) AS cnt_reviews
    FROM books
         LEFT JOIN ratings ON ratings.book_id = books.book_id
         LEFT JOIN reviews ON reviews.book_id = books.book_id
GROUP BY books.title, books.book_id
ORDER BY cnt_reviews DESC, avg_rating DESC
   LIMIT 5;                    
        
        '''
get(query)

,title,avg_rating,cnt_reviews
0,Twilight (Twilight #1),3.66,7
1,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),4.41,6
2,Harry Potter and the Chamber of Secrets (Harry Potter #2),4.29,6
3,The Book Thief,4.26,6
4,The Glass Castle,4.21,6


При сортировке по количеству обзоров и среднему рейтингу в топ-5 вошли:

1. Сумерки - 7 обзоров, средний рейтинг 3.66;
2. Гарри Поттер (3 часть) - 6 обзоров, средний рейтинг 4.41;
3. Гарри Поттер (2 часть) - 6 обзоров, средний рейтинг 4.29;
4. Книжный вор - 6 обзоров, средний рейтинг 4.26;
5. Замок из стекла - 6 обзоров, средний рейтинг 4.21

### Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query = '''

  SELECT b.publisher_id, p.publisher, count(*) AS count_books
    FROM books AS b
         LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
   WHERE num_pages > 50
GROUP BY b.publisher_id, p.publisher
ORDER BY count(*) DESC
   LIMIT 1;
        
        '''
get(query)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Наибольшее число книг толще 50 страниц выпустило издательство `Penguin Books` - 42 книги.

### Определить автора с самой высокой средней оценкой книг (среди книг с 50 и более оценками)

In [12]:
query = '''

    WITH rat AS (
                  SELECT book_id, avg(rating) AS ratings
                    FROM ratings
                GROUP BY book_id
                  HAVING count(*) > 50
             )
  SELECT a.author, avg(ratings)
    FROM rat 
    JOIN books AS b ON rat.book_id = b.book_id 
    JOIN authors AS a ON b.author_id = a.author_id
GROUP BY a.author
ORDER BY avg(ratings) DESC;
        
        '''
get(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
8,William Shakespeare/Paul Werstine/Barbara A. Mowat,3.79
9,Dan Brown,3.75


Наивысшая средняя оценка книг у Д. Роулинг. Это довольно ожидаемый результат - серия книг о Гарри Поттере повсеместно пользуется популярностью. Средняя оценка ее книг в БД нашего сервиса - 4.28 балла.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query = '''

SELECT avg(count_reviews)
  FROM (
          SELECT username, count(*) AS count_reviews
            FROM reviews
           WHERE username
                 IN (
                          SELECT username
                            FROM ratings
                        GROUP BY username
                          HAVING count(*) > 50
                    )
        GROUP BY username
       ) AS rev;
        
        '''
get(query)

,avg
0,24.33


В БД присутствует информация об активных пользователях, оставивших более 50 оценок. В среднем такие пользователи оставляют 24.8 обзоров на книгу.

## Общие выводы


В нашей базе:
- 1000 уникальных книг на английском языке;
- от 636 авторов;
- от 340 издательств;
- 6456 оценок, что довольно неплохо для такого объема книг, в среднем по 6 оценок на книгу;
- 2793 обзора.

В результате исследования базы мы выяснили:
    
- В БД нового сервиса содержится информация о 1000 книг
- Порядка 82% книг выпущены за последние 20 лет
- Для большинства книг в БД есть оценки и обзоры

При сортировке по количеству обзоров и среднему рейтингу в топ-5 вошли:
1. Сумерки - 7 обзоров, средний рейтинг 3.66;
2. Гарри Поттер (3 часть) - 6 обзоров, средний рейтинг 4.41;
3. Гарри Поттер (2 часть) - 6 обзоров, средний рейтинг 4.29;
4. Книжный вор - 6 обзоров, средний рейтинг 4.26;
5. Замок из стекла - 6 обзоров, средний рейтинг 4.21

- Наибольшее количество книг в нашей подборки выпущены издательством "Penguin Books" - 42 штуки, что составляет 4% от нашей подборки
- Самый высокие оценки у автора Д. Роулинг - 4.3
- В базе есть информация об активных пользователях - купленный сервис достаточно "живой", но располагает не слишком большой подборкой книг.